# TCC-II

## Aluno: Dylan Faria Robson
## Turma: SI8N
## Período: 2024/2



# Código

## Bibliotecas

In [1]:
#Para uso no Google Colab
#!pip install datasets
#!pip install seqeval
#!pip install evaluate

In [ ]:
import torch
from transformers import AutoTokenizer, AutoTokenizer, AutoModelForTokenClassification, Trainer, TrainingArguments, AutoModelForSequenceClassification, pipeline, AutoModelForQuestionAnswering, AdamW
from torch.utils.data import DataLoader
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, classification_report
from datasets import load_dataset
from evaluate import load
from tqdm.auto import tqdm
from inspect import isfunction
import PySimpleGUI as sg
import warnings
warnings.filterwarnings('ignore')

## Opcional

In [3]:
#Opções de temas do PySimpleGUI
#sg.theme_previewer()

In [ ]:
#Configurando PysimpleGUI
sg.theme('DarkTeal9') #Tema padrão
##sg.theme('DarkPurple4') -> Tema alternativo

#Baixe o icone no link: https://github.com/MrRobson9/TCC-II/blob/main/Main/Icon.ico 
icon_path='./icon.ico' #Mesma pasta do Notebook
sg.set_options(icon=icon_path)

## Funções Customizadas

Grupo de funções para ser utilizadas com modelos cutomizados.

In [5]:
def obter_entradas_usuario_gererica():
    """
  Exibe uma série de interfaces gráficas para o usuário fornecer as entradas necessárias para a avaliação de um modelo customizado.

    A função conduz o usuário por três seções distintas:
    1. Configuração do modelo, onde o usuário nomeia o modelo e dataset e escolhe a atividade a ser avaliada.
    2. Definição das funções, onde o usuário fornece os nomes das funções que retornarão o modelo e o dataset, além de uma função opcional de tokenização.
    3. Configuração da avaliação, onde o usuário indica o nome da função que avaliará o modelo com o dataset.

    Durante a interação, os campos são validados, e o usuário é guiado para a próxima seção somente após fornecer todas as informações obrigatórias.

    return:
        - nome (str): Nome do modelo customizado fornecido pelo usuário.
        - atividade (str): Tipo de atividade (NER, Sentiment Analysis, Q&A) selecionada pelo usuário.
        - nome_funcao_modelo (str): Nome da função que retorna o modelo carregado.
        - nome_funcao_dataset (str): Nome da função que retorna o dataset carregado.
        - opcao_tokenizador (str): Indica se o dataset precisa ser tokenizado ("1" para Sim, "2" para Não).
        - nome_tokenizador (str): Nome da função de tokenização (se aplicável).
        - nome_eval (str): Nome da função que realiza a avaliação do modelo com o dataset.
        - nome_dataset (str): Nome do dataset fornecido pelo usuário.
    """
    def janela_modelo():
        custom_intro_text = (
        "Essa sessão tem como objetivo ajudar na avaliação de um modelo personalizado.\n"
        "Para isso será necessario, no minimo, uma função que retorne um modelo já treinado, uma outra que retorne o dataset que será utilizado e uma função de avaliação. \nÉ opcional a utilização de pré-processamento (atente-se a necessidade do seu modelo e dataset)\n"
        "____________________________________________________________________________\n"
        "\nPrimeiramente escolha um nome para o seu modelo e dataset. Essa será a forma que eles apareceram na tabela com as metricas.\n"
    )

    # Layout da janela
        layout_modelo = [
            [sg.Multiline(custom_intro_text, size=(80, 10), disabled=True, font=("Helvetica", 10))],
            [sg.Text("Digite o nome do modelo (como será apresentado na tabela de métricas):", font=('Helvetica', 12), text_color='white')],
            [sg.InputText(key="nome", font=('Helvetica', 10))],
            
            [sg.Text("Qual atividade será avaliada (escolha um número):", font=('Helvetica', 12), text_color='white')],
            [sg.Radio("NER", "atividade", key="atividade_NER"), 
             sg.Radio("Análise de Sentimentos", "atividade", key="atividade_SENTIMENT"), 
             sg.Radio("Q&A", "atividade", key="atividade_QA")],
            
            [sg.Text("Nome do dataset (como aparecerá na tabela de métricas):", font=('Helvetica', 12), text_color='white')],
            [sg.InputText(key="nome_dataset", font=('Helvetica', 10))],
            
            [sg.Text("", key="error_modelo", text_color='red')],
            [sg.Button("Próximo", key="Proximo_modelo", button_color=('white', 'green'))]
        ]
        
        window = sg.Window('Configuração do Modelo Customizado', layout_modelo, size=(600, 400), finalize=True, resizable=True)
        return window

    # Função para exibir a janela da seção "Funções"
    def janela_funcoes():
        custom_funcoes_text = (
            "Agora você deve ter salvo em memoria as funções:\n"
            "1 - Uma função que seja chamada sem argumentos e retorne um modelo como 'criar_modelo_1'\n"
            "2 - Uma função que seja chamada sem argumentos e retorne o dataset como 'carregar_dataset_A'\n"
            "3 (Opcional) - Uma função que seja chamada passando o dataset como argumento e retorne\n"
            "sua versão processada como 'tokenizacao' que será utilizado como 'tokenizacao(dataset).\n" 

        )
        layout_funcoes = [
            [sg.Multiline(custom_funcoes_text, size=(80, 7), disabled=True, font=("Helvetica", 10))],
            [sg.Text("Nome da função que retorna o modelo carregado:", font=('Helvetica', 12), text_color='white')],
            [sg.InputText(key="nome_funcao_modelo", font=('Helvetica', 10))],
            
            [sg.Text("Nome da função que retorna o dataset carregado:", font=('Helvetica', 12), text_color='white')],
            [sg.InputText(key="nome_funcao_dataset", font=('Helvetica', 10))],
            
            [sg.Text("O dataset precisará passar pelo processo de pré-processamento?", font=('Helvetica', 12), text_color='white')],
            [sg.Radio("Sim", "opcao_tokenizador", key="token_sim", enable_events=True), 
             sg.Radio("Não", "opcao_tokenizador", key="token_nao", enable_events=True)],
            
            [sg.Text("Nome da função de pré-processamento:", key="label_tokenizador", visible=False, font=('Helvetica', 12), text_color='white')],
            [sg.InputText(key="nome_tokenizador", visible=False, font=('Helvetica', 10))],
            
            [sg.Text("", key="error_funcoes", text_color='red')],
            [sg.Button("Próximo", key="Proximo_funcoes", button_color=('white', 'green'))]
        ]
        
        window = sg.Window('Configuração das Funções Customizadas', layout_funcoes, size=(600, 450), finalize=True, resizable=True)
        return window

    # Função para exibir a janela da seção "Avaliação"
    def janela_avaliacao():
        custom_avaliacao_text = (
            "Por fim, deve ser fornecido o nome de uma função que receba como parametros o modelo e dataset (nessa ordem) e retorne um pandas.Series ou dicionario contendo 'eval_accuracy', 'eval_precision', 'eval_recall' e 'eval_f1'.\nEx.: 'avaliar_modelo' que será utilizado como 'avaliar_modelo(modelo, dataset)'."
            )

        layout_avaliacao = [
            [sg.Multiline(custom_avaliacao_text, size=(80, 5), disabled=True, font=("Helvetica", 10))],
            [sg.Text("Nome da função para Avaliar o modelo:", font=('Helvetica', 12), text_color='white')],
            [sg.InputText(key="nome_eval", font=('Helvetica', 10))],
            
            [sg.Text("", key="error_avaliacao", text_color='red')],
            [sg.Button("Enviar", key="Enviar", button_color=('white', 'green'))]
        ]
        
        window = sg.Window('Configuração da Avaliação Customizada', layout_avaliacao, size=(600, 400), finalize=True, resizable=True)
        return window

    # Função principal para controlar o fluxo de janelas
    def controle_janelas():
        # Mostra a primeira janela
        window = janela_modelo()
        while True:
            event, values = window.read()
            window.BringToFront()
            if event in (sg.WIN_CLOSED, "Cancelar"):
                window.close()
                raise Exception("O usuario interrompeu o processo ou fechou a janela.")

            # Valida e controla a navegação para a próxima janela
            if event == "Proximo_modelo":
                nome = values.get("nome")
                atividade = "NER" if values.get("atividade_NER") else "SENTIMENT" if values.get("atividade_SENTIMENT") else "Q&A" if values.get("atividade_QA") else None
                nome_dataset = values.get("nome_dataset")
                
                if not nome or not atividade or not nome_dataset:
                    window["error_modelo"].update("Todos os campos são obrigatórios.")
                else:
                    window.close()
                    window = janela_funcoes()
                    
                    while True:
                        event, values = window.read()
                        window.BringToFront()
                        if event in (sg.WIN_CLOSED, "Cancelar"):
                            window.close()
                            raise Exception("O usuario interrompeu o processo ou fechou a janela.")

                        # Mostrar ou esconder campo de tokenizador
                        if values.get("token_sim"):
                            window["label_tokenizador"].update(visible=True)
                            window["nome_tokenizador"].update(visible=True)
                        elif values.get("token_nao"):
                            window["label_tokenizador"].update(visible=False)
                            window["nome_tokenizador"].update(visible=False)

                        # Valida e controla a navegação para a próxima janela
                        if event == "Proximo_funcoes":
                            nome_funcao_modelo = values.get("nome_funcao_modelo")
                            nome_funcao_dataset = values.get("nome_funcao_dataset")
                            opcao_tokenizador = "1" if values.get("token_sim") else "2"
                            nome_tokenizador = values.get("nome_tokenizador") if opcao_tokenizador == "1" else "Default"

                            if not nome_funcao_modelo or not nome_funcao_dataset:
                                window["error_funcoes"].update("Todos os campos são obrigatórios.")
                            elif opcao_tokenizador == "Sim" and not nome_tokenizador:
                                window["error_funcoes"].update("O nome da função para pré-processamento é obrigatório.")
                            else:
                                window.close()
                                window = janela_avaliacao()
                                
                                while True:
                                    event, values = window.read()
                                    window.BringToFront()
                                    if event in (sg.WIN_CLOSED, "Cancelar"):
                                        window.close()
                                        raise Exception("O usuario interrompeu o processo ou fechou a janela.")

                                    # Valida e processa os dados ao clicar em "Enviar"
                                    if event == "Enviar":
                                        nome_eval = values.get("nome_eval")
                                        
                                        if not nome_eval:
                                            window["error_avaliacao"].update("O nome da função para avaliação é obrigatório.")
                                        else:
                                            window.close()
                                            # Retorna os valores coletados
                                            return (nome, 
                                                    atividade, 
                                                    nome_funcao_modelo, 
                                                    nome_funcao_dataset, 
                                                    opcao_tokenizador, 
                                                    nome_tokenizador, 
                                                    nome_eval, 
                                                    nome_dataset)

    return controle_janelas()

In [6]:
def carregar_funcao_generico(nome_funcao):
    """
    Carrega e executa uma função global com o nome fornecido.

    A função busca uma função pelo nome dentro do escopo global e a executa. Se o nome não corresponder a uma função, 
    é lançada uma exceção do tipo `TypeError`. A função retornada é então executada sem argumentos e seu resultado é retornado.

    Parameters:
        - nome_funcao (str): O nome da função que deve ser carregada e executada.

    return:
        - result (any): O resultado da execução da função especificada.

    Raises:
        - TypeError: Se o nome fornecido não corresponder a uma função global.
    """

    funcao = globals()[nome_funcao]
    if not isfunction(funcao):
        raise TypeError(f"O nome da função {nome_funcao} está incorreto ou não é uma função.")
    result = funcao()
    return result

In [7]:
def pre_processamento_dataset_generico(dataset, opcao_tokenizador, nome_tokenizador):
    """
    Aplica a tokenização a um dataset com base nas opções fornecidas.

    A função verifica se a opção de tokenização foi selecionada. Se sim, ela tenta carregar e executar a função de tokenização especificada 
    pelo nome. Se a função de tokenização não for encontrada ou não for uma função válida, uma exceção do tipo `TypeError` é lançada. 
    Caso a tokenização não seja necessária, a função retorna o dataset original.

    Parameters:
        - dataset (any): O dataset que pode ser tokenizado.
        - opcao_tokenizador (str): Indica se a tokenização deve ser aplicada ("1" para Sim, "2" para Não).
        - nome_tokenizador (str): O nome da função de tokenização a ser utilizada se a opção for "1".

    return:
        - tokenized_datasets (any): O dataset tokenizado, ou o dataset original se a tokenização não for aplicada.

    Raises:
        - TypeError: Se o nome da função de tokenização fornecido estiver incorreto ou não for uma função.
    """
    if opcao_tokenizador == "1":
        if not isfunction(nome_tokenizador):
          nome_tokenizador = globals()[nome_tokenizador]
        if isfunction(nome_tokenizador):
           tokenized_datasets = nome_tokenizador(dataset)
        else:
          raise TypeError(f"O nome da função de pré-processamento {nome_tokenizador} está incorreto ou não é uma função.")
        tokenized_datasets = nome_tokenizador(dataset)
    else:
        tokenized_datasets = dataset
    return tokenized_datasets

In [8]:
def avaliar_modelo_generico(nome_eval, modelo, tokenized_datasets):
    """
    Executa a avaliação de um modelo utilizando uma função de avaliação especificada.

    A função busca a função de avaliação pelo nome dentro do escopo global e a executa, passando como parâmetros o modelo e o dataset.
    Se o nome fornecido não corresponder a uma função, uma exceção do tipo `TypeError` é lançada.

    Parameters:
        - nome_eval (str): O nome da função que realizará a avaliação do modelo.
        - modelo (any): O modelo que será avaliado.
        - tokenized_datasets (any): O dataset que será utilizado na avaliação.

    return:
        - result (any): O resultado da função de avaliação aplicada ao modelo e ao dataset.

    Raises:
        - TypeError: Se o nome da função de avaliação fornecido estiver incorreto ou não for uma função.
    """
    if not isfunction(nome_eval):
        nome_eval = globals()[nome_eval]
        
    if isfunction(nome_eval):
        result = nome_eval(modelo, tokenized_datasets)
        return result
    else:
        raise TypeError(f"O nome da função de avaliação {nome_eval} está incorreto ou não é uma função")
        

In [9]:
def atualizar_resultados(arquivo_resultados, nome, atividade, nome_dataset, eval_results):
    """
    Atualiza o arquivo CSV de resultados com as métricas de avaliação de um modelo.

    A função carrega o arquivo CSV de resultados e atualiza ou adiciona as métricas de avaliação de um modelo específico. 
    Se o modelo já existir no arquivo e a nova avaliação tiver um `f1-score` superior ao existente, os valores de precisão, 
    recall, acurácia e `f1-score` são atualizados. Se o modelo não existir, um novo registro é adicionado. 
    Os resultados são então ordenados por atividade, `f1-score`, acurácia, precisão, recall e nome do modelo antes de serem salvos de volta no arquivo.

    Parameters:
        - arquivo_resultados (str): O caminho para o arquivo CSV onde os resultados são armazenados.
        - nome (str): O nome do modelo avaliado.
        - atividade (str): A atividade (NER, Sentiment, Q&A, etc.) associada ao modelo.
        - nome_dataset (str): O nome do dataset utilizado na avaliação.
        - eval_results (dict ou pd.Series): Um dicionário contendo as métricas de avaliação, incluindo `eval_accuracy`, `eval_precision`, 
          `eval_recall`, e `eval_f1`.

    return:
        - df_resultados (pd.DataFrame): O DataFrame atualizado com os resultados das avaliações.

    Raises:
        - FileNotFoundError: Se o arquivo CSV de resultados não for encontrado, um novo arquivo é criado.
    """
    try:
        df_resultados = pd.read_csv(arquivo_resultados)
    except FileNotFoundError:
        df_resultados = pd.DataFrame(columns=["Modelo", "Atividade", "Dataset", "accuracy", "precision", "recall", "f1-score"])

    if nome in df_resultados["Modelo"].values:
        idx = df_resultados[df_resultados["Modelo"] == nome].index[0]
        if eval_results['eval_f1'] > df_resultados.at[idx, "f1-score"]:
            df_resultados.at[idx, "accuracy"] = eval_results['eval_accuracy']
            df_resultados.at[idx, "precision"] = eval_results['eval_precision']
            df_resultados.at[idx, "recall"] = eval_results['eval_recall']
            df_resultados.at[idx, "f1-score"] = eval_results['eval_f1']
    else:
        novo_resultado = {
            "Modelo": nome,
            "Atividade": atividade,
            "Dataset": nome_dataset,
            "accuracy": round(eval_results['eval_accuracy'], 3),
            "precision": round(eval_results['eval_precision'], 3),
            "recall": round(eval_results['eval_recall'], 3),
            "f1-score": round(eval_results['eval_f1'], 3)
        }
        df_resultados = df_resultados._append(novo_resultado, ignore_index=True)

    df_resultados = df_resultados.sort_values(by=["Atividade", "f1-score", "accuracy", "precision", "recall", "Modelo"], ascending=[True, False, False, False, False, True])
    df_resultados.to_csv(arquivo_resultados, index=False)
    return df_resultados


In [10]:
def avaliacao_generica_customizada(arquivo_resultados="resultados.csv"):
    """
    Realiza a avaliação personalizada de um modelo utilizando funções genéricas fornecidas pelo usuário.

    A função conduz o processo de avaliação de um modelo personalizado, solicitando ao usuário as entradas necessárias 
    através de uma interface gráfica. Em seguida, carrega o modelo e o dataset especificado, realiza a tokenização do 
    dataset (se necessário) e avalia o modelo. Finalmente, atualiza um arquivo CSV com os resultados da avaliação.

    Parameters:
        - arquivo_resultados (str): O caminho para o arquivo CSV onde os resultados serão armazenados. 
          O valor padrão é "resultados.csv".

    return:
        - df_resultados (pd.DataFrame): O DataFrame atualizado contendo os resultados da avaliação.

    Process Overview:
        1. Coleta as entradas do usuário utilizando a função `obter_entradas_usuario_gererica()`.
        2. Carrega o modelo e o dataset utilizando funções especificadas pelo usuário.
        3. Tokeniza o dataset, se necessário, com base nas entradas do usuário.
        4. Avalia o modelo utilizando uma função de avaliação fornecida pelo usuário.
        5. Atualiza o arquivo CSV com os resultados da avaliação, ordenando os modelos com base em suas métricas.

    Exceptions:
        - TypeError: Levantado se ocorrer um erro ao registrar as respostas do usuário ou se uma função especificada 
          não for válida.
        - RuntimeError: Levantado se ocorrer um erro durante o carregamento do modelo, dataset, tokenização ou avaliação.

    Raises:
        - TypeError: Se as entradas do usuário ou as funções fornecidas estiverem incorretas.
        - RuntimeError: Se o modelo, dataset, tokenização ou avaliação falharem.
    """

    nome, atividade, nome_funcao_modelo, nome_funcao_dataset, opcao_tokenizador, nome_tokenizador, nome_eval, nome_dataset = obter_entradas_usuario_gererica()
    if nome == None or atividade == None or nome_funcao_modelo == None or nome_funcao_dataset == None or opcao_tokenizador == None or nome_tokenizador == None or nome_eval == None:
      
      raise TypeError("Houve um erro ao registrar as respostas do usuario.")

    modelo = carregar_funcao_generico(nome_funcao_modelo)
    if modelo == None:
      raise RuntimeError("Houve um erro ao carregar o modelo.")

    dataset = carregar_funcao_generico(nome_funcao_dataset)
    if dataset == None:
      raise RuntimeError("Houve um erro ao carregar o dataset.")
    tokenized_datasets = pre_processamento_dataset_generico(dataset, opcao_tokenizador, nome_tokenizador)
    if tokenized_datasets == None:
      raise RuntimeError("Houve um erro ao tokenizar o dataset.")
    
    eval_results = avaliar_modelo_generico(nome_eval, modelo, tokenized_datasets)
    if type(eval_results) == type(None):
      raise RuntimeError("Houve um erro ao avaliar o modelo.")

    df_resultados = atualizar_resultados(arquivo_resultados, nome, atividade, nome_dataset, eval_results)
    if type(df_resultados) == type(None):
      raise RuntimeError("Houve um erro ao gerar a tabela com metricas.")

    return df_resultados

## Funções Transformers Padrão

Funções que utilizam como base modelos diponibilizados a partir da biblioteca transformers e o hub de modelos do Hugging Face.

In [11]:
def obter_entradas_usuario_transformers():
    """
    Exibe uma interface gráfica para o usuário fornecer as entradas necessárias para a avaliação de modelos Transformers.
    
    A função permite que o usuário selecione o nome de um modelo existente e o dataset correspondente a partir de opções predefinidas,
    utilizando listas suspensas (combos). Os dados coletados são então retornados para uso posterior.

    return:
        - nome (str): Nome do modelo selecionado pelo usuário.
        - opcao_dataset (int): Opção de dataset escolhida pelo usuário.
    """
    # Definição das opções de modelos e datasets

    transformers_text = (
        'Essa sessão permitirá a escolha de um modelo e dataset contemplado.\n'
        "OBS. Para adicionar um modelo e dataset padrão, refira-se à seção de funções 'Funções Transformers Padrão'."
    )

    modelos = ["BERT_NER", "GPT_NER", "DISTILBERT_NER", "DISTILBERT_SENTIMENT", "GPT_SENTIMENT", "ALBERT_SENTIMENT", 
               "DISTILBERT_Q&A", "BERT_BASE_Q&A", "BERT_LARGE_Q&A"]
    datasets = ["conll2003 - NER", "IMDB - Sentiment", "SQUaD - Q&A"]

    # Layout da janela
    layout = [
        [sg.Multiline(transformers_text, size=(80, 4), disabled=True, font=("Helvetica", 10))],
        [sg.Text("Avaliação Padrão", font=("Helvetica", 16))],
        [sg.Text("1 - Selecione o nome do modelo existente")],
        [sg.Combo(modelos, key="nome", readonly=True, size=(30, 1))],
        [sg.Text("2 - Selecione o dataset correspondente à atividade do modelo")],
        [sg.Combo(datasets, key="opcao_dataset", readonly=True, size=(30, 1))],
        [sg.Button("Enviar"), sg.Button("Cancelar")]
    ]

    # Criação da janela
    window = sg.Window("Avaliação de Modelos Transformers", layout)

    # Loop de eventos para interação com o usuário
    while True:
        event, values = window.read()
        window.BringToFront()

        # Fechar a janela se o usuário clicar em "Cancelar" ou fechar a janela
        if event == sg.WIN_CLOSED or event == "Cancelar":
            window.close()
            raise Exception("O usuario interrompeu o processo ou fechou a janela.")

        # Verificar se o usuário preencheu ambos os campos
        if event == "Enviar":
            if values["nome"] and values["opcao_dataset"]:
                nome = values["nome"]
                opcao_dataset = datasets.index(values["opcao_dataset"]) + 1
                window.close()
                return nome, opcao_dataset
            else:
                sg.popup("Por favor, selecione o nome do modelo e o dataset antes de prosseguir.")

In [12]:

def criar_modelo(nome):
    """
    Cria e retorna um modelo fine-tuned e a atividade associada com base no nome fornecido.

    A função aceita um nome de modelo específico, que é utilizado para carregar um modelo fine-tuned 
    correspondente da biblioteca Hugging Face. Com base no modelo selecionado, a função também retorna 
    a atividade correspondente, como NER, análise de sentimentos ou Q&A. Se o nome do modelo fornecido 
    não for encontrado, uma exceção será levantada.

    Parameters:
        - nome (str): O nome do modelo fine-tuned a ser carregado. 

    Returns:
        - modelo (transformers.PreTrainedModel): O modelo pré-treinado correspondente.
        - atividade (str): A atividade associada ao modelo (NER, SENTIMENT, ou Q&A).

    Raises:
        - ValueError: Se o nome do modelo não for encontrado nas opções disponíveis.
    """
    # Criação de modelo e atividade
    if nome == "BERT_NER":
      modelo = AutoModelForTokenClassification.from_pretrained("MrRobson9/bert-base-cased-finetuned-conll2003-english-ner")
      atividade = "NER"
    elif nome == "GPT_NER":
      modelo = AutoModelForTokenClassification.from_pretrained("MrRobson9/gpt2-ner-conll2003-english")
      atividade = "NER"
    elif nome == "DISTILBERT_NER":
      modelo = AutoModelForTokenClassification.from_pretrained("MrRobson9/distilbert-base-cased-finetuned-conll2003-english-ner")
      atividade = "NER"
    elif nome == "DISTILBERT_SENTIMENT":
      modelo = AutoModelForSequenceClassification.from_pretrained("distilbert/distilbert-base-uncased-finetuned-sst-2-english")
      atividade = "SENTIMENT"
    elif nome == "GPT_SENTIMENT":
      modelo = AutoModelForSequenceClassification.from_pretrained("michelecafagna26/gpt2-medium-finetuned-sst2-sentiment")
      atividade = "SENTIMENT"
    elif nome == "ALBERT_SENTIMENT":
      modelo = AutoModelForSequenceClassification.from_pretrained("Alireza1044/albert-base-v2-sst2")
      atividade = "SENTIMENT"
    elif nome == "DISTILBERT_Q&A":
      modelo = AutoModelForQuestionAnswering.from_pretrained("distilbert-base-cased-distilled-squad")
      atividade = "Q&A"
    elif nome == "BERT_BASE_Q&A":
      modelo = AutoModelForQuestionAnswering.from_pretrained("csarron/bert-base-uncased-squad-v1")
      atividade = "Q&A"
    elif nome == "BERT_LARGE_Q&A":
      modelo = AutoModelForQuestionAnswering.from_pretrained("google-bert/bert-large-uncased-whole-word-masking-finetuned-squad")
      atividade = "Q&A"
    else:
        raise ValueError(f"\nModelo {nome} não encontrado. Verifique as opções e tente novamente.")
    return modelo, atividade

In [13]:
def escolha_dataset(atividade, escolha):
  """
    Seleciona e carrega um dataset pré-definido com base na atividade e escolha fornecidas.

    A função recebe a atividade (como NER, análise de sentimentos ou Q&A) e uma escolha numérica 
    que indica qual dataset carregar. Com base nessas informações, a função carrega o dataset 
    correspondente usando a biblioteca Hugging Face `datasets` e retorna o dataset e seu nome. 
    Se a combinação de atividade e escolha for inválida, uma exceção será levantada.

    Parameters:
        - atividade (str): A atividade associada ao modelo, como "NER", "SENTIMENT", ou "Q&A".
        - escolha (int): Um número inteiro que representa a escolha do dataset. Os valores esperados são:
            - 1 para "conll2003" (NER)
            - 2 para "imdb" (SENTIMENT)
            - 3 para "squad" (Q&A)

    Returns:
        - dataset (datasets.DatasetDict): O dataset carregado correspondente à atividade e escolha fornecidas.
        - nome_dataset (str): O nome do dataset carregado.

    Raises:
        - ValueError: Se a combinação de escolha e atividade for inválida ou não suportada.
    """

  #Criação do dataset e nome
  if escolha == 1 and atividade == "NER":
    dataset = load_dataset("conll2003", trust_remote_code=True)
    nome_dataset = "conll2003"
  elif escolha == 2 and atividade == "SENTIMENT":
    dataset = load_dataset("imdb", trust_remote_code=True)
    nome_dataset = "imdb"
  elif escolha == 3 and atividade == "Q&A":
    dataset = load_dataset("squad", trust_remote_code=True)
    nome_dataset = "squad"

  else: #Exceção
    raise ValueError("\nOpção inválida ou dataset incompatível com a atividade escolhida/modelo.")
  return dataset, nome_dataset

In [14]:
def escolha_pre_processamento(nome, dataset):
  """
    Seleciona e aplica o pré-processamento adequado ao dataset com base no nome do modelo.

    A função recebe o nome do modelo e o dataset, determinando o tokenizador correto e o método de 
    pré-processamento correspondente. Dependendo do modelo, a função pode aplicar o alinhamento de 
    rótulos (`tokenize_and_align_labels`) para tarefas de NER, ou a tokenização genérica (`tokenize_function`) 
    para tarefas de análise de sentimentos e Q&A. O dataset é então mapeado através dessas funções de 
    pré-processamento e retornado como um conjunto de dados tokenizado.

    Parameters:
        - nome (str): O nome do modelo a ser utilizado. Os valores esperados são:
            - "BERT_NER", "GPT_NER", "DISTILBERT_NER" para tarefas de NER.
            - "DISTILBERT_SENTIMENT", "GPT_SENTIMENT", "ALBERT_SENTIMENT" para tarefas de análise de sentimentos.
            - "DISTILBERT_Q&A", "BERT_BASE_Q&A", "BERT_LARGE_Q&A" para tarefas de perguntas e respostas.
        - dataset (datasets.DatasetDict): O dataset que será pré-processado.

    Returns:
        - tokenized_datasets (datasets.DatasetDict): O dataset após a aplicação do pré-processamento adequado.
    
    Raises:
        - TypeError: Se o tokenizador não for encontrado para o nome do modelo fornecido.
  """
  #Modelos que utilizam tokenize_and_align_labels()
  if nome in ["BERT_NER", "GPT_NER", "DISTILBERT_NER"]:
    if nome == "BERT_NER":
      tokenizer = AutoTokenizer.from_pretrained("MrRobson9/bert-base-cased-finetuned-conll2003-english-ner")
    elif nome == "GPT_NER":
      tokenizer = AutoTokenizer.from_pretrained("MrRobson9/gpt2-ner-conll2003-english")
    elif nome == "DISTILBERT_NER":
      tokenizer = AutoTokenizer.from_pretrained("MrRobson9/distilbert-base-cased-finetuned-conll2003-english-ner")

    def tokenize_and_align_labels(examples):
      tokenized_inputs = tokenizer(examples['tokens'], truncation=True, padding='max_length', is_split_into_words=True, max_length=128)
      labels = []
      for i, label in enumerate(examples['ner_tags']):
          word_ids = tokenized_inputs.word_ids(batch_index=i)
          previous_word_idx = None
          label_ids = []
          for word_idx in word_ids:
              if word_idx is None:
                  label_ids.append(-100)
              elif word_idx != previous_word_idx:
                  label_ids.append(label[word_idx])
              else:
                  label_ids.append(-100)
              previous_word_idx = word_idx
          labels.append(label_ids)
      tokenized_inputs["labels"] = labels
      return tokenized_inputs
    tokenized_datasets = dataset.map(tokenize_and_align_labels, batched=True)


  #Modelos que utilizam tokenize_function() - função genérica
  if nome in ["DISTILBERT_SENTIMENT", "GPT_SENTIMENT", "ALBERT_SENTIMENT"]:
    if nome == "DISTILBERT_SENTIMENT":
      tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased-finetuned-sst-2-english")
    elif nome == "GPT_SENTIMENT":
      tokenizer = AutoTokenizer.from_pretrained("michelecafagna26/gpt2-medium-finetuned-sst2-sentiment")
    elif nome == "ALBERT_SENTIMENT":
      tokenizer = AutoTokenizer.from_pretrained("Alireza1044/albert-base-v2-sst2")

    def tokenize_function(examples):
      return tokenizer(examples["text"], padding="max_length", truncation=True)
    tokenized_datasets = dataset.map(tokenize_function, batched=True)

  if nome in ["DISTILBERT_Q&A", "BERT_BASE_Q&A", "BERT_LARGE_Q&A"]:
    if nome == "DISTILBERT_Q&A":
      model_name = "distilbert-base-cased-distilled-squad"
    elif nome == "BERT_BASE_Q&A":
      model_name = "csarron/bert-base-uncased-squad-v1"
    elif nome == "BERT_LARGE_Q&A":
      model_name = "google-bert/bert-large-uncased-whole-word-masking-finetuned-squad"
    tokenized_datasets = AutoTokenizer.from_pretrained(model_name) #Tokenizador
    tokenizer = tokenized_datasets

    #Adicionar novos modelos Aqui

  if tokenizer == None: #Exceção
    raise TypeError(f"\nTokenizador {nome} não encontrado.")
  return tokenized_datasets

In [15]:
def avaliar_modelo(modelo, tokenized_datasets, atividade, dataset, nome, nome_dataset, arquivo_resultados="resultados.csv"):
    """
    Avalia um modelo de NLP com base na atividade especificada (NER, SENTIMENT, Q&A) e 
    atualiza os resultados em um arquivo CSV.

    A função avalia o desempenho do modelo usando métricas apropriadas para a atividade indicada. Dependendo 
    da atividade, são utilizadas diferentes abordagens de avaliação:
    - NER: Usa métricas de NER (SeqEval - precisão, recall, F1 e acurácia) e valida o modelo com o dataset de validação.
    - SENTIMENT: Usa métricas de precisão, recall, F1 e acurácia para análise de sentimentos.
    - Q&A: Avalia a precisão das respostas em uma tarefa de Perguntas e Respostas (Q&A) e calcula métricas de 
      precisão, recall, F1 e acurácia.

    Parameters:
        - modelo (transformers.PreTrainedModel): O modelo a ser avaliado.
        - tokenized_datasets (datasets.DatasetDict): O dataset tokenizado para avaliação.
        - atividade (str): O tipo de atividade para a avaliação. Pode ser "NER", "SENTIMENT" ou "Q&A".
        - dataset (datasets.DatasetDict): O dataset original utilizado na avaliação.
        - nome (str): Nome do modelo para identificar e armazenar os resultados.
        - nome_dataset (str): Nome do dataset utilizado na avaliação.
        - arquivo_resultados (str): Caminho para o arquivo CSV onde os resultados serão salvos. O padrão é 
          "resultados.csv".

    Returns:
        - pd.DataFrame: Um DataFrame com os resultados atualizados da avaliação, incluindo acurácia, precisão, recall e F1-score.

    Raises:
        - ValueError: Se a atividade especificada não for válida.
        - FileNotFoundError: Se o arquivo CSV não for encontrado ou não puder ser criado.
    """
    if atividade == "NER":
      if nome in ["BERT_NER", "GPT_NER", "DISTILBERT_NER"]:
        metric = load("seqeval", trust_remote_code=True)
        label_list = tokenized_datasets['validation'].features['ner_tags'].feature.names

        #Definir a função de avaliação
        def compute_metrics(p):
            predictions, labels = p
            predictions = np.argmax(predictions, axis=2)

            true_labels = [[label_list[l] for l in label if l != -100] for label in labels]
            true_predictions = [
                [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
                for prediction, label in zip(predictions, labels)
            ]

            results = metric.compute(predictions=true_predictions, references=true_labels)
            return {
                "accuracy": results["overall_accuracy"],
                "precision": results["overall_precision"],
                "recall": results["overall_recall"],
                "f1": results["overall_f1"],
            }

        #Configurações de treinamento
        training_args = TrainingArguments(
                output_dir = "./results",
                eval_strategy="epoch",
                per_device_eval_batch_size=16 if nome in ["BERT_NER", "DISTILBERT_NER"] else 8
            )

        #Inicializar o Trainer
        trainer = Trainer(
            model=modelo,
            args=training_args,
            eval_dataset=tokenized_datasets["validation"],
            compute_metrics=compute_metrics,
        )

        #Avaliar o modelo
        eval_results = trainer.evaluate()

    elif atividade == "SENTIMENT":
      accuracy_metric = load("accuracy")
      precision_metric = load("precision")
      recall_metric = load("recall")
      f1_metric = load("f1")

      #Definir a função de avaliação
      def compute_metrics(p):
          predictions, labels = p
          preds = np.argmax(predictions, axis=1)

          accuracy = accuracy_metric.compute(predictions=preds, references=labels)
          precision = precision_metric.compute(predictions=preds, references=labels, average='binary')
          recall = recall_metric.compute(predictions=preds, references=labels, average='binary')
          f1 = f1_metric.compute(predictions=preds, references=labels, average='binary')

          return {
              "accuracy": accuracy["accuracy"],
              "precision": precision["precision"],
              "recall": recall["recall"],
              "f1": f1["f1"],
          }
      #Configurações de treinamento
      if nome == "DISTILBERT_SENTIMENT" or nome == "GPT_SENTIMENT" or nome == "ALBERT_SENTIMENT":
        training_args = TrainingArguments(
            output_dir="./results",
            evaluation_strategy="epoch",
            per_device_eval_batch_size=8,
        )


      #Inicializar o Trainer
      trainer = Trainer(
          model=modelo,
          args=training_args,
          eval_dataset=tokenized_datasets["test"],
          compute_metrics=compute_metrics,
      )

      #Avaliar o modelo
      eval_results = trainer.evaluate()

    elif atividade == "Q&A":
      if nome == "DISTILBERT_Q&A" or nome == "BERT_BASE_Q&A" or nome == "BERT_LARGE_Q&A":
        qa_pipeline = pipeline("question-answering", model=modelo, tokenizer=tokenized_datasets, device=0)
        #Definir a função de avaliação
        def evaluate_model(dataset, qa_pipeline):
          true_answers = []
          predicted_answers = []

          for example in tqdm(dataset['validation'], desc="Avaliando", unit="it/s"):
              context = example["context"]
              question = example["question"]
              answers = example["answers"]["text"]

              result = qa_pipeline(question=question, context=context)
              predicted_answer = result["answer"]

              # Verificar se a resposta prevista está entre as respostas corretas
              if predicted_answer in answers:
                  true_answers.append(predicted_answer)
              else:
                  true_answers.append(answers[0])  # Utilizar a primeira resposta correta

              predicted_answers.append(predicted_answer)

          precision = precision_score(true_answers, predicted_answers, average='weighted')
          recall = recall_score(true_answers, predicted_answers, average='weighted')
          f1 = 2 * (precision * recall) / (precision + recall)
          accuracy = accuracy_score(true_answers, predicted_answers)

          eval_results = pd.Series({ 'eval_accuracy': accuracy, 'eval_precision': precision, 'eval_recall':recall, 'eval_f1': f1})


          return eval_results

        eval_results = evaluate_model(dataset, qa_pipeline)

    else: #Exceção
        raise ValueError(f"\nOpção {atividade} inválida.")

    #Carregar resultados existentes
    try:
        df_resultados = pd.read_csv(arquivo_resultados)
    except FileNotFoundError:
        df_resultados = pd.DataFrame(columns=["Modelo", "Atividade", "Dataset", "accuracy", "precision", "recall", "f1-score"])

    #Verificar se o modelo já foi avaliado
    if nome in df_resultados["Modelo"].values:
        idx = df_resultados[df_resultados["Modelo"] == nome].index[0]
        if eval_results['eval_f1'] > df_resultados.at[idx, "f1-score"]:
            df_resultados.at[idx, "accuracy"] = eval_results['eval_accuracy']
            df_resultados.at[idx, "precision"] = eval_results['eval_precision']
            df_resultados.at[idx, "recall"] = eval_results['eval_recall']
            df_resultados.at[idx, "f1-score"] = eval_results['eval_f1']
    else:
        novo_resultado = {
            "Modelo": nome,
            "Atividade": atividade,
            "Dataset": nome_dataset,
            "accuracy": round(eval_results['eval_accuracy'], 3),
            "precision": round(eval_results['eval_precision'], 3),
            "recall": round(eval_results['eval_recall'], 3),
            "f1-score": round(eval_results['eval_f1'], 3)
        }
        df_resultados = df_resultados._append(novo_resultado, ignore_index=True)

    #Ordenar resultados
    df_resultados = df_resultados.sort_values(by=["Atividade", "f1-score", "accuracy", "precision", "recall", "Modelo"], ascending=[True, False, False, False, False, True])
    #Salvar resultados atualizados
    df_resultados.to_csv(arquivo_resultados, index=False)
    return df_resultados

In [16]:
def avaliacao_transformers_padrao():
    """
    Executa uma avaliação padrão para modelos Transformers, abrangendo desde a criação do modelo até a avaliação e 
    armazenamento dos resultados.

    Esta função coordena o processo de avaliação de um modelo Transformer seguindo os seguintes passos:
    1. Obtém as entradas do usuário, incluindo o nome do modelo e a opção de dataset.
    2. Cria o modelo apropriado e define a atividade (NER, SENTIMENT ou Q&A) associada.
    3. Escolhe e carrega o dataset correspondente à atividade e à opção selecionada.
    4. Realiza o pré-processamento dos dados utilizando a função de tokenização adequada para o modelo e a atividade.
    5. Avalia o modelo usando o dataset tokenizado e calcula métricas de desempenho.
    6. Atualiza e retorna a tabela de métricas de avaliação.

    Returns:
        - pd.DataFrame: Um DataFrame com os resultados da avaliação, incluindo métricas como acurácia, precisão, recall e F1-score.

    Raises:
        - RuntimeError: Se ocorrer um erro durante a criação do modelo, escolha do dataset, pré-processamento ou avaliação.
    """
    #Nome do modelo
    nome, opcao_dataset = obter_entradas_usuario_transformers()

    # Instanciamento do modelo e escolha de atividade
    modelo, atividade = criar_modelo(nome)
    if modelo == None:
        raise RuntimeError("Houve um erro ao criar o modelo.")

    # Escolha do dataset
    dataset, nome_dataset = escolha_dataset(atividade, opcao_dataset)
    if dataset == None:
        raise RuntimeError("Houve um erro ao escolher o dataset.")

    # Escolha de função de pré-processamento
    tokenized_datasets = escolha_pre_processamento(nome, dataset)
    if tokenized_datasets == None:
        raise RuntimeError("Houve um erro ao escolher o pre-processamento.")

    # Avaliação do modelo
    tabela_metricas = avaliar_modelo(modelo, tokenized_datasets, atividade, dataset, nome, nome_dataset)
    if type(tabela_metricas) !=  type(None):
        return tabela_metricas
    else:
        raise RuntimeError("Houve um erro ao criar a tabela de métricas.")


## Custom

### Exemplo

Funções de exemplo para serem utilizadas em conjunto com "Funções Customizads"

In [17]:
def criar_modelo_generico():
    """
    Cria e carrega um modelo Transformer genérico para tarefas de tokenização, junto com seu otimizador.

    Esta função realiza os seguintes passos:
    1. Carrega um modelo pré-treinado de classificação de tokens (DistilBERT) com o número especificado de rótulos.
    2. Configura o dispositivo para o treinamento, utilizando GPU se disponível.
    3. Cria um otimizador AdamW com uma taxa de aprendizado padrão.
    4. Carrega os estados do modelo e do otimizador a partir de arquivos pré-salvos.
    5. Move o modelo para o dispositivo configurado (GPU ou CPU).

    Returns:
        - model (AutoModelForTokenClassification): O modelo Transformer carregado e preparado para uso.
    """
    model_path = input('Qual o caminho do modelo Pytorch?\nR: ') #exemplo: C:\Users\usuario\pasta\modelo\DistilBERT\best_model.pth
    optimizer_path = input('Qual o caminho do optimizer?\nR: ') #exemplo: C:\Users\usuario\pasta\modelo\DistilBERT\optimizer.pth
    model = AutoModelForTokenClassification.from_pretrained("distilbert-base-uncased", num_labels=9)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    optimizer = AdamW(model.parameters(), lr=5e-5)
    model.load_state_dict(torch.load(model_path, map_location=torch.device(device)))
    optimizer.load_state_dict(torch.load(optimizer_path, map_location=torch.device(device)))
    model.to(device)
    return model

In [18]:
def carregar_dataset_conll2003():
    """
    Carrega o dataset 'conll2003' e o formata para uso com PyTorch.

    Esta função realiza os seguintes passos:
    1. Define o nome do dataset como "conll2003".
    2. Carrega o dataset usando a função `load_dataset` da biblioteca `datasets`.
    3. Converte o dataset para o formato "torch", que é compatível com modelos PyTorch.

    Returns:
        - dataset (Dataset): O dataset 'conll2003' carregado e formatado para uso com PyTorch.
    """
    nome_dataset = "conll2003" #Change to a input later
    dataset = load_dataset(nome_dataset, trust_remote_code=True).with_format("torch")
    return dataset

In [19]:
def tokenizer_generico(dataset):
    """
    Tokeniza o dataset fornecido usando o tokenizador DistilBERT e prepara um DataLoader para o conjunto de validação.

    Esta função realiza os seguintes passos:
    1. Inicializa o tokenizador `distilbert-base-uncased`.
    2. Define a função `tokenize_and_align_labels` para tokenizar e alinhar as etiquetas com as entradas do modelo.
    3. Aplica a função de tokenização ao dataset, retornando um dataset tokenizado.
    4. Define uma função `collate_fn` para agrupar as amostras em lotes para o DataLoader.
    5. Cria um DataLoader para o conjunto de validação do dataset tokenizado.

    Parameters:
        - dataset (DatasetDict): O dataset a ser tokenizado, que deve conter um conjunto de validação.

    Returns:
        - val_loader: Um DataLoader PyTorch para o conjunto de validação, com amostras agrupadas e preparadas para treinamento.
    """
    nome_tokenizer = "distilbert-base-uncased" #Change to a input later
    tokenizer = AutoTokenizer.from_pretrained(nome_tokenizer)
    def tokenize_and_align_labels(examples):
      tokenized_inputs = tokenizer(examples['tokens'], truncation=True, padding='max_length', is_split_into_words=True, max_length=128, return_tensors='pt')
      labels = []
      for i, label in enumerate(examples['ner_tags']):
          word_ids = tokenized_inputs.word_ids(batch_index=i)
          previous_word_idx = None
          label_ids = []
          for word_idx in word_ids:
              if word_idx is None:
                  label_ids.append(-100)
              elif word_idx != previous_word_idx:
                  label_ids.append(label[word_idx])
              else:
                  label_ids.append(-100)
              previous_word_idx = word_idx
          labels.append(label_ids)
      tokenized_inputs["labels"] = torch.tensor(labels)
      return tokenized_inputs

    # Tokenizar o dataset
    tokenized_datasets = dataset.map(tokenize_and_align_labels, batched=True)

    def collate_fn(batch):
      input_ids = torch.stack([item['input_ids'].squeeze() for item in batch])
      attention_mask = torch.stack([item['attention_mask'].squeeze() for item in batch])
      labels = torch.stack([item['labels'].squeeze() for item in batch])
      return {'input_ids': input_ids, 'attention_mask': attention_mask, 'labels': labels}

    val_dataset = tokenized_datasets["validation"]
    val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False, collate_fn=collate_fn)

    return val_loader

In [20]:
def avaliacao_generica(model, val_loader):
    """
    Avalia o modelo usando o DataLoader de validação e calcula métricas de desempenho.

    Esta função realiza os seguintes passos:
    1. Prepara o modelo e o dispositivo para a avaliação.
    2. Avalia o modelo no conjunto de validação, acumulando as previsões e etiquetas verdadeiras.
    3. Calcula a perda média de avaliação e as métricas de desempenho (precisão, recall, F1-score e acurácia) usando o relatório de classificação.

    Parameters:
        - model (PreTrainedModel): O modelo a ser avaliado.
        - val_loader (DataLoader): O DataLoader PyTorch contendo o conjunto de validação.

    Returns:
        - pd.Series: Um objeto Series do pandas contendo as métricas de avaliação: acurácia, precisão, recall e F1-score.
    """
    label_list =  val_loader.dataset.features['ner_tags'].feature.names
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.eval()
    eval_loss = 0
    all_predictions = []
    all_labels = []
    with torch.no_grad():
        for batch in tqdm(val_loader, desc="Avaliando"):
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            eval_loss += loss.item()
            predictions = outputs.logits.argmax(dim=-1)
            for i in range(len(labels)):
                true_labels = [label_list[l] for l in labels[i].tolist() if l != -100]
                true_predictions = [label_list[p] for (p, l) in zip(predictions[i].tolist(), labels[i].tolist()) if l != -100]
                all_labels.extend(true_labels)
                all_predictions.extend(true_predictions)
            eval_loss = eval_loss / len(val_loader)

    report = classification_report(all_labels, all_predictions, output_dict=True, zero_division=0)
    precision = report['weighted avg']['precision']
    recall = report['weighted avg']['recall']
    f1 = report['weighted avg']['f1-score']
    accuracy = report['accuracy']
    eval_results = pd.Series({ 'eval_accuracy': accuracy, 'eval_precision': precision, 'eval_recall':recall, 'eval_f1': f1})
    return eval_results

## Main

Funções principais que utilizam as anteriores internamente.

In [21]:
def escolher_opcao_avaliacao():
    """
    Exibe uma interface gráfica para o usuário escolher a opção de avaliação.

    A função apresenta informações sobre o framework e permite que o usuário escolha entre
    modelos padrão ou um modelo customizado para realizar a avaliação. Dependendo da escolha,
    a função correspondente será chamada para realizar a avaliação e retornar uma tabela com métricas.

    Returns:
        - tabela_metricas: Tabela com métricas, dependendo da avaliação escolhida.
    """

    # Texto de introdução e instruções
    intro_text = (
        "Bem-vindo a plataforma para avaliação de modelos NLP.\n"
        "Versão Atual: v1.0\n"
        "Esse trabalho é parte integrante do Trabalho de Conclusão de Curso do autor Dylan Faria Robson, "
        "sob orientação do professor Jean-Remi Bourguet no curso de Sistemas de Informação na Universidade "
        "Vila Velha (ES).\n"
        "O objetivo deste é auxiliar na avaliação de modelos de NLP.\n"
        "Escolha se você quer utilizar um modelo e dataset já inplementado ou escolher um customizado por você.\n"
        "Lembre-se, essa plataforma tem o propósito de ajudar, porém podem ser necessárias alterações para que "
        "se adeque ao seu cenário e necessidades.\n"
    )

    # Layout da janela
    layout = [
        [sg.Multiline(intro_text, size=(80, 10), disabled=True, font=("Helvetica", 10))],
        [sg.Text("Você gostaria de proceder com os modelos padrões ou modelo customizado?", font=("Helvetica", 12))],
        [sg.Radio("Modelos padrões", "RADIO1", default=True, key="padrao")],
        [sg.Radio("Modelo customizado", "RADIO1", key="customizado")],
        [sg.Button("Enviar"), sg.Button("Cancelar")]
    ]

    # Criação da janela
    window = sg.Window("Escolha de Opção de Avaliação", layout)
    
    # Loop de eventos para interação com o usuário
    while True:
        event, values = window.read()
        window.BringToFront()
        # Fechar a janela se o usuário clicar em "Cancelar" ou fechar a janela
        if event == sg.WIN_CLOSED or event == "Cancelar":
            window.close()
            raise Exception("O usuario interrompeu o processo ou fechou a janela.")

        # Ação a ser tomada dependendo da escolha do usuário
        if event == "Enviar":
            window.close()  # Fecha a janela imediatamente após clicar em "Enviar"

            if values["padrao"]:
                tabela_metricas = avaliacao_transformers_padrao()
            elif values["customizado"]:
                tabela_metricas = avaliacao_generica_customizada()
            else:
                sg.popup("Por favor, escolha uma das opções de avaliação.")
                continue

            return tabela_metricas


In [22]:
def plataforma_avaliacao_NLP():
    """
    Executa o processo de avaliação de NLP, escolhendo a opção de avaliação e gerando a tabela de métricas.

    Esta função realiza os seguintes passos:
    1. Solicita ao usuário que escolha uma opção de avaliação.
    2. Gera uma tabela de métricas com base na escolha do usuário.
    3. Se a tabela de métricas for gerada com sucesso, exibe uma mensagem de sucesso e retorna a tabela.
    4. Se ocorrer um erro durante a geração da tabela, levanta uma exceção.

    Returns:
        pd.DataFrame: Uma tabela de métricas contendo as avaliações do modelo, se a geração for bem-sucedida.

    Raises:
        RuntimeError: Se ocorrer um erro ao gerar a tabela de métricas.
    """    
    #Escolha de avaliação
    tabela_metricas = escolher_opcao_avaliacao()

    if type(tabela_metricas) != type(None):
      print("\nTabela de metricas gerada com sucesso!\n")
      return tabela_metricas
    else: #Exceção
      raise RuntimeError("\nErro ao gerar a tabela de metricas.")


## Uso

Chamada da função prinicpal - plataforma_avaliacao_NLP - para exibir a interface grafica desenvolvida.

In [ ]:
plataforma_avaliacao_NLP()

## Modelos e Datasets

Modelos Utilizados:
* https://huggingface.co/distilbert/distilbert-base-uncased
* https://huggingface.co/google-bert/bert-base-uncased
* https://huggingface.co/openai-community/gpt2
* https://huggingface.co/albert/albert-base-v2

Avaliados:
* https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english
* https://huggingface.co/michelecafagna26/gpt2-medium-finetuned-sst2-sentiment
* https://huggingface.co/Alireza1044/albert-base-v2-sst2
* https://huggingface.co/distilbert/distilbert-base-cased-distilled-squad
* https://huggingface.co/csarron/bert-base-uncased-squad-v1
* https://huggingface.co/google-bert/bert-large-uncased-whole-word-masking-finetuned-squad

Datasets:
* https://huggingface.co/datasets/stanfordnlp/imdb
* https://huggingface.co/datasets/eriktks/conll2003
* https://huggingface.co/datasets/rajpurkar/squad
* https://huggingface.co/datasets/stanfordnlp/sst2